In [1]:
import pandas as pd
import psycopg2

def execute_query(sql_query, dbname='temp', user='postgres', password='postgres', port='5432'):
    # Create a connection to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, port=port)

    # Use read_sql to execute the query and load the results into a DataFrame
    df = pd.read_sql(sql_query, conn)

    # Close the database connection
    conn.close()

    # Return the DataFrame
    return df


# ISO 8601

Which date format below conforms to the ISO 8601 standard?

- 2018-06-15 15:30:00

# Date comparisons

When working with timestamps, sometimes you want to find all observations on a given day. However, if you specify only a date in a comparison, you may get unexpected results. This query:
```
SELECT count(*) 
  FROM evanston311
 WHERE date_created = '2018-01-02';
 ```
returns 0, even though there were 49 requests on January 2, 2018.

This is because dates are automatically converted to timestamps when compared to a timestamp. The time fields are all set to zero:
```
SELECT '2018-01-02'::timestamp;
 2018-01-02 00:00:00
 ```

In [2]:

query_result = execute_query("""
-- Count requests created on January 31, 2017
SELECT count(*) 
  FROM evanston311
 WHERE date_created::date = '2017-01-31';                    
""")
query_result

,count
0,45


In [3]:

query_result = execute_query("""
-- Count requests created on February 29, 2016
SELECT count(*)
  FROM evanston311 
 WHERE date_created >= '2016-02-29' 
   AND date_created < '2016-02-29'::DATE + '1 day'::INTERVAL;                  
""")
query_result

,count
0,55


In [4]:

query_result = execute_query("""
-- Count requests created on March 13, 2017
SELECT count(*)
  FROM evanston311
 WHERE date_created >= '2017-03-13'
   AND date_created < '2017-03-13'::date + 1;               
""")
query_result

,count
0,33


# Date arithmetic

You can subtract dates or timestamps from each other.

You can add time to dates or timestamps using intervals. An interval is specified with a number of units and the name of a datetime field.

In [5]:

query_result = execute_query("""
-- Subtract the min date_created from the max
SELECT MAX(date_created)-MIN(date_created)
  FROM evanston311;          
""")
query_result

,?column?
0,911 days 16:33:39


In [6]:

query_result = execute_query("""
-- How old is the most recent request?
SELECT NOW() - MAX(date_created)
  FROM evanston311;   
""")
query_result

,?column?
0,2057 days 12:34:21.984833


In [7]:

query_result = execute_query("""
-- Add 100 days to the current timestamp
SELECT NOW() + '100 days'::INTERVAL;
""")
query_result

,?column?
0,2024-05-27 05:10:58.390318+00:00


In [8]:

query_result = execute_query("""
-- Select the current timestamp, 
-- and the current timestamp + 5 minutes
SELECT NOW(), NOW()+ '5 minutes'::INTERVAL;
""")
query_result

,now,?column?
0,2024-02-17 05:11:14.591606+00:00,2024-02-17 05:16:14.591606+00:00


# Completion time by category

The evanston311 data includes a `date_created` timestamp from when each request was created and a `date_completed` timestamp for when it was completed. The difference between these tells us how long a request was open.

In [9]:

query_result = execute_query("""
-- Select the category and the average completion time by category
SELECT category, 
       AVG(date_completed - date_created) AS completion_time
  FROM evanston311
 GROUP BY category
-- Order the results
 ORDER BY completion_time DESC;
""")
query_result

,category,completion_time
0,Rodents- Rats,64 days 10:58:23.000766
1,Fire Prevention - Public Education,34 days 16:48:10
2,Key Request - All City Employees,32 days 00:52:11
3,Smoking,27 days 07:42:30.238095
4,Notice of Violation,24 days 05:41:12.666667
...,...,...
144,Child Seat Installation or Inspection,0 days 08:37:10.424124
145,Animal Service - Wilmette,0 days 08:10:43.500000
146,Lawn Irrigation Violation,0 days 07:02:40.666667
147,"Food Waste, Collective Resource- Ask a Questio...",0 days 03:33:31


# Date parts

The `date_part()` function is useful when you want to aggregate data by a unit of time across multiple larger units of time. For example, aggregating data by month across different years, or aggregating by hour across different days.

In [11]:

query_result = execute_query("""
-- Extract the month from date_created and count requests
SELECT DATE_PART( 'month', date_created) AS month, 
       COUNT(*)
  FROM evanston311
 -- Limit the date range
 WHERE date_created >= '2016-01-01'
   AND date_created < '2018-01-01'
 -- Group by what to get monthly counts?
 GROUP BY month
 ORDER BY month;
""")
query_result

,month,count
0,1.0,1808
1,2.0,1775
2,3.0,2173
3,4.0,2382
4,5.0,2672
5,6.0,3402
6,7.0,3062
7,8.0,3108
8,9.0,2767
9,10.0,2399


In [12]:

query_result = execute_query("""
-- Get the hour and count requests
SELECT DATE_PART('hour' , date_created) AS hour,
       count(*)
  FROM evanston311
 GROUP BY hour
 -- Order results to select most common
 ORDER BY count DESC
 LIMIT 1;
""")
query_result

,hour,count
0,15.0,4089


In [13]:

query_result = execute_query("""
-- Count requests completed by hour
SELECT DATE_PART('hour' ,date_completed ) AS hour,
       COUNT(*)
  FROM evanston311
 GROUP BY hour
 ORDER BY hour DESC;
""")
query_result

,hour,count
0,23.0,428
1,22.0,1605
2,21.0,5242
3,20.0,5059
4,19.0,4787
5,18.0,3580
6,17.0,3351
7,16.0,3162
8,15.0,2744
9,14.0,2516


# Variation by day of week

Does the time required to complete a request vary by the day of the week on which the request was created?

In [14]:

query_result = execute_query("""
-- Select name of the day of the week the request was created 
SELECT TO_CHAR( date_created,'day' ) AS day, 
       -- Select avg time between request creation and completion
       AVG(date_completed - date_created) AS duration
  FROM evanston311 
 -- Group by the name of the day of the week and 
 -- integer value of day of week the request was created
 GROUP BY day, EXTRACT(DOW FROM date_created)::INTEGER
 -- Order by integer value of the day of the week 
 -- the request was created
 ORDER BY EXTRACT(DOW FROM date_created)::INTEGER;
""")
query_result

,day,duration
0,sunday,7 days 05:23:48.742129
1,monday,7 days 01:02:52.921913
2,tuesday,7 days 05:58:14.733105
3,wednesday,7 days 10:41:22.482254
4,thursday,7 days 11:36:04.049344
5,friday,8 days 03:16:58.234490
6,saturday,8 days 13:11:56.032318


# Date truncation

Unlike `date_part()` or `EXTRACT()`, `date_trunc() `keeps date/time units larger than the field you specify as part of the date. So instead of just extracting one component of a timestamp, date_trunc() returns the specified unit and all larger ones as well.

In [15]:

query_result = execute_query("""
-- Aggregate daily counts by month
SELECT DATE_TRUNC('month',day) AS month,
       AVG(count)
  -- Subquery to compute daily counts
  FROM (SELECT DATE_TRUNC('day',date_created) AS day,
               COUNT(*) AS count
          FROM evanston311
         GROUP BY day) AS daily_count
 GROUP BY month
 ORDER BY month;
""")
query_result

,month,avg
0,2015-12-31 18:00:00+00:00,24.233333
1,2016-01-31 18:00:00+00:00,30.689655
2,2016-02-29 18:00:00+00:00,35.645161
3,2016-03-31 18:00:00+00:00,37.200000
4,2016-04-30 18:00:00+00:00,39.483871
5,2016-05-31 18:00:00+00:00,43.833333
6,2016-06-30 18:00:00+00:00,41.677419
7,2016-07-31 18:00:00+00:00,46.419355
8,2016-08-31 18:00:00+00:00,47.466667
9,2016-09-30 18:00:00+00:00,35.806452


# Find missing dates

Are there any days in the Evanston 311 data where no requests were created?

In [16]:

query_result = execute_query("""
SELECT day
-- 1) Subquery to generate all dates
-- from min to max date_created
  FROM (SELECT generate_series(MIN(date_created),
                               MAX(date_created),
                               '1 day')::date AS day
          -- What table is date_created in?
          FROM evanston311) AS all_dates
-- 4) Select dates (day from above) that are NOT IN the subquery
 WHERE day NOT IN 
       -- 2) Subquery to select all date_created values as dates
       (SELECT date_created::date
          FROM evanston311);
""")
query_result

,day
0,2016-01-10
1,2016-12-11
2,2017-03-12
3,2018-01-07


In [17]:

query_result = execute_query("""
-- Generate 6 month bins covering 2016-01-01 to 2018-06-30

-- Create lower bounds of bins
SELECT generate_series('2016-01-01',  -- First bin lower value
                       '2018-01-01',  -- Last bin lower value
                       '6 months'::interval) AS lower,
-- Create upper bounds of bins
       generate_series('2016-07-01',  -- First bin upper value
                       '2018-07-01',  -- Last bin upper value
                       '6 months'::interval) AS upper;
""")
query_result

,lower,upper
0,2015-12-31 18:00:00+00:00,2016-06-30 18:00:00+00:00
1,2016-06-30 18:00:00+00:00,2016-12-31 18:00:00+00:00
2,2016-12-31 18:00:00+00:00,2017-06-30 18:00:00+00:00
3,2017-06-30 18:00:00+00:00,2017-12-31 18:00:00+00:00
4,2017-12-31 18:00:00+00:00,2018-06-30 18:00:00+00:00


In [18]:

query_result = execute_query("""
-- Count number of requests made per day
SELECT day, COUNT(id) AS count
-- Use a daily series from 2016-01-01 to 2018-06-30 
-- to include days with no requests
  FROM (SELECT generate_series('2016-01-01',  -- series start date
                               '2018-06-30',  -- series end date
                               '1 day'::interval)::date AS day) AS daily_series
       LEFT JOIN evanston311
       -- match day from above (which is a date) to date_created
       ON day = date_created::date
 GROUP BY day;
""")
query_result

,day,count
0,2016-01-01,5
1,2016-01-02,21
2,2016-01-03,12
3,2016-01-04,55
4,2016-01-05,31
...,...,...
907,2018-06-26,56
908,2018-06-27,41
909,2018-06-28,69
910,2018-06-29,53


In [28]:

query_result = execute_query("""
-- Bins from Step 1
WITH bins AS (
	 SELECT generate_series('2016-01-01',
                            '2018-01-01',
                            '6 months'::interval) AS lower,
            generate_series('2016-07-01',
                            '2018-07-01',
                            '6 months'::interval) AS upper),
-- Daily counts from Step 2
     daily_counts AS (
     SELECT day, count(date_created) AS count
       FROM (SELECT generate_series('2016-01-01',
                                    '2018-06-30',
                                    '1 day'::interval)::date AS day) AS daily_series
            LEFT JOIN evanston311
            ON day = date_created::date
      GROUP BY day)
-- Select bin bounds 
SELECT lower, 
       upper, 
       -- Compute median of count for each bin
       percentile_disc(0.5) WITHIN GROUP (ORDER BY count) AS median
  -- Join bins and daily_counts
  FROM bins
       LEFT JOIN daily_counts
       -- Where the day is between the bin bounds
       ON day >= lower
          AND day < upper
 -- Group by bin bounds
 GROUP BY lower, upper
 ORDER BY lower;
""")
query_result

,lower,upper,median
0,2015-12-31 18:00:00+00:00,2016-06-30 18:00:00+00:00,37
1,2016-06-30 18:00:00+00:00,2016-12-31 18:00:00+00:00,41
2,2016-12-31 18:00:00+00:00,2017-06-30 18:00:00+00:00,44
3,2017-06-30 18:00:00+00:00,2017-12-31 18:00:00+00:00,51
4,2017-12-31 18:00:00+00:00,2018-06-30 18:00:00+00:00,41


# Monthly average with missing dates

Find the average number of Evanston 311 requests created per day for each month of the data.

This time, do not ignore dates with no requests.

In [20]:

query_result = execute_query("""
-- generate series with all days from 2016-01-01 to 2018-06-30
WITH all_days AS 
     (SELECT generate_series('2016-01-01',
                             '2018-06-30',
                             '1 day'::INTERVAL) AS date),
     -- Subquery to compute daily counts
     daily_count AS 
     (SELECT date_trunc('day', date_created) AS day,
             count(*) AS count
        FROM evanston311
       GROUP BY day)
-- Aggregate daily counts by month using date_trunc
SELECT DATE_TRUNC('month',all_days.date ) AS month,
       -- Use coalesce to replace NULL count values with 0
       avg(coalesce(daily_count.count, 0)) AS average
  FROM all_days
       LEFT JOIN daily_count
       -- Joining condition
       ON all_days.date=daily_count.day
 GROUP BY month
 ORDER BY month; 
""")
query_result

,month,average
0,2015-12-31 18:00:00+00:00,23.451613
1,2016-01-31 18:00:00+00:00,30.689655
2,2016-02-29 18:00:00+00:00,35.645161
3,2016-03-31 18:00:00+00:00,37.200000
4,2016-04-30 18:00:00+00:00,39.483871
5,2016-05-31 18:00:00+00:00,43.833333
6,2016-06-30 18:00:00+00:00,41.677419
7,2016-07-31 18:00:00+00:00,46.419355
8,2016-08-31 18:00:00+00:00,47.466667
9,2016-09-30 18:00:00+00:00,35.806452


# Longest gap

What is the longest time between Evanston 311 requests being submitted?

In [22]:

query_result = execute_query("""
-- Compute the gaps
WITH request_gaps AS (
        SELECT date_created,
               -- lead or lag
               LAG(date_created) OVER (ORDER BY date_created) AS previous,
               -- compute gap as date_created minus lead or lag
               date_created - LAG(date_created) OVER (ORDER BY date_created) AS gap
          FROM evanston311)
-- Select the row with the maximum gap
SELECT *
  FROM request_gaps
-- Subquery to select maximum gap from request_gaps
 WHERE gap = (SELECT MAX(gap)
                FROM request_gaps);
""")
query_result

,date_created,previous,gap
0,2018-01-07 18:41:34+00:00,2018-01-05 18:04:09+00:00,2 days 00:37:25


# Rats!

Requests in category "Rodents- Rats" average over 64 days to resolve. Why?

Investigate in 4 steps:

- Why is the average so high? Check the distribution of completion times. Hint: `date_trunc()` can be used on intervals.
- See how excluding outliers influences average completion times.
- Do requests made in busy months take longer to complete? Check the correlation between the average completion time and requests per month.
- Compare the number of requests created per month to the number completed.

In [23]:

query_result = execute_query("""
-- Truncate the time to complete requests to the day
SELECT DATE_TRUNC('day' ,date_completed - date_created ) AS completion_time,
-- Count requests with each truncated time
       COUNT(*)
  FROM evanston311
-- Where category is rats
 WHERE category = 'Rodents- Rats'
-- Group and order by the variable of interest
 GROUP BY completion_time
 ORDER BY COUNT;
""")
query_result

,completion_time,count
0,315 days,1
1,232 days,1
2,280 days,1
3,190 days,1
4,161 days,1
...,...,...
204,44 days,19
205,2 days,23
206,12 days,23
207,13 days,42


In [25]:

query_result = execute_query("""
SELECT category, 
       -- Compute average completion time per category
       AVG(date_completed - date_created) AS avg_completion_time
  FROM evanston311
-- Where completion time is less than the 95th percentile value
 WHERE (date_completed - date_created) < 
-- Compute the 95th percentile of completion time in a subquery
         (SELECT percentile_disc(0.95) WITHIN GROUP (ORDER BY date_completed - date_created)
            FROM evanston311)
 GROUP BY category
-- Order the results
 ORDER BY avg_completion_time DESC;
""")
query_result

,category,avg_completion_time
0,"Trash Cart - Downsize, Upsize or Remove",12 days 17:47:50.586912
1,Sanitation Billing Questions,12 days 11:13:25.888889
2,THIS REQUEST IS INACTIVE...Trash Cart - Compos...,12 days 06:32:42.024390
3,"Trash, Recycling, Yard Waste Cart- Repair/Repl...",11 days 18:48:27.488108
4,Rodents- Rats,11 days 08:58:00.840849
...,...,...
143,Child Seat Installation or Inspection,0 days 08:37:10.424124
144,Animal Service - Wilmette,0 days 08:10:43.500000
145,Lawn Irrigation Violation,0 days 07:02:40.666667
146,"Food Waste, Collective Resource- Ask a Questio...",0 days 03:33:31


In [26]:

query_result = execute_query("""
-- Compute correlation (corr) between 
-- avg_completion time and count from the subquery
SELECT CORR(avg_completion, count)
  -- Convert date_created to its month with date_trunc
  FROM (SELECT DATE_TRUNC('month', date_created) AS month, 
               -- Compute average completion time in number of seconds           
               AVG(EXTRACT(epoch FROM date_completed - date_created)) AS avg_completion, 
               -- Count requests per month
               count(*) AS count
          FROM evanston311
         -- Limit to rodents
         WHERE category='Rodents- Rats' 
         -- Group by month, created above
         GROUP BY month) 
         -- Required alias for subquery 
         AS monthly_avgs;
""")
query_result

,corr
0,0.248855


In [27]:

query_result = execute_query("""
-- Compute monthly counts of requests created
WITH created AS (
       SELECT DATE_TRUNC('month' ,date_created) AS month,
              count(*) AS created_count
         FROM evanston311
        WHERE category='Rodents- Rats'
        GROUP BY month),
-- Compute monthly counts of requests completed
      completed AS (
       SELECT DATE_TRUNC('month' ,date_completed ) AS month,
              count(*) AS completed_count
         FROM evanston311
        WHERE category='Rodents- Rats'
        GROUP BY month)
-- Join monthly created and completed counts
SELECT created.month, 
       created_count, 
       completed_count
  FROM created
       INNER JOIN completed
       ON created.month=completed.month
 ORDER BY created.month;
""")
query_result

,month,created_count,completed_count
0,2015-12-31 18:00:00+00:00,10,1
1,2016-01-31 18:00:00+00:00,22,11
2,2016-02-29 18:00:00+00:00,31,14
3,2016-03-31 18:00:00+00:00,36,16
4,2016-04-30 18:00:00+00:00,40,19
5,2016-05-31 18:00:00+00:00,41,49
6,2016-06-30 18:00:00+00:00,80,47
7,2016-07-31 18:00:00+00:00,79,43
8,2016-08-31 18:00:00+00:00,56,58
9,2016-09-30 18:00:00+00:00,76,67
